# DIN-SQL을 활용한 자연어-SQL 변환: 고급 프롬프팅 기법

안녕하세요! 이 노트북에서는 자연어 질문을 데이터베이스에서 실행 가능한 SQL 쿼리로 변환하는 고급 기법을 배워보겠습니다. 

## 이 노트북을 통해 배울 수 있는 것
- DIN-SQL이라는 혁신적인 프롬프팅 전략
- 자연어 질문을 SQL로 변환하는 체계적인 접근법
- AI 모델과 데이터베이스를 연결하여 실제 데이터를 조회하는 방법
- 복잡한 쿼리를 단계별로 분해하여 처리하는 방법

## 배경 지식
SQL(Structured Query Language)은 데이터베이스에서 데이터를 조회, 삽입, 수정, 삭제하는 표준 언어입니다. 하지만 일반 사용자가 자연어 질문을 SQL로 변환하는 것은 쉽지 않습니다. DIN-SQL은 이 문제를 해결하기 위해 개발된 방법론입니다.

---

## 실습 환경 권장사항
이 노트북을 실행하기 위한 권장 환경입니다.

**Amazon SageMaker를 사용하는 경우:**
- SageMaker 이미지: `sagemaker-distribution-cpu`
- 커널: `Python 3`
- 인스턴스 타입: `ml.m5.large`

**참고:** 이 설정은 비용 효율적이면서도 충분한 성능을 제공합니다.

---

## 목차

1. [의존성 설치](#step-1-install-dependencies)
1. [Athena 연결 설정](#step-2-set-up-connection-to-the-tpc-ds-data-set-in-athena)
1. [스키마 연결](#step-3-determine-schema-links)
1. [쿼리 복잡도 분류](#step-4-classify-sql-complexity)
1. [SQL 쿼리 생성](#step-5-generate-sql-query)
1. [SQL 쿼리 실행](#step-6-execute-query)
1. [결과 검증](#step-7-validate-results)
1. [자기교정](#step-8-self-correction)
1. [실험](#step-9-experiment)
1. [참고문헌](#citation)

---

## 학습 목표
이 노트북을 완료하면 다음을 할 수 있게 됩니다:

1. **자연어 질문을 SQL로 변환하는 체계적인 방법 이해**
   - "어떤 고객이 가장 많은 돈을 썼나요?"와 같은 질문을 SQL 쿼리로 변환

2. **DIN-SQL 방법론의 4단계 프로세스 실습**
   - 스키마 연결 (Schema Linking)
   - 복잡도 분류 (Classification)  
   - SQL 생성 (Generation)
   - 자기교정 (Self-correction)

3. **실제 데이터베이스와 AI 모델을 연결하여 동작하는 시스템 구축**

## 왜 이것이 중요한가요?
- 비개발자도 데이터베이스에서 필요한 정보를 쉽게 찾을 수 있습니다
- 복잡한 SQL 쿼리 작성 시간을 크게 단축할 수 있습니다
- AI와 데이터베이스를 연결하는 실용적인 방법을 배울 수 있습니다

---

## DIN-SQL 방법론 소개

### DIN-SQL이란?
**DIN-SQL(Decomposed In-Context Learning of Text-to-SQL)**은 자연어 질문을 SQL 쿼리로 변환하는 혁신적인 접근 방법입니다. 복잡한 문제를 작은 단위로 나누어 해결하는 "분할 정복" 전략을 사용합니다.

### 4단계 프로세스
DIN-SQL은 다음 4단계로 구성됩니다:

1. **스키마 연결 (Schema Linking)** 
   - 질문에 답하기 위해 어떤 테이블과 컬럼이 필요한지 파악
   - 예: "고객"과 "판매" 정보가 필요하다면 customer 테이블과 sales 테이블을 연결

2. **분류 및 분해 (Classification and Decomposition)**
   - 질문의 복잡도를 분석 (쉬움/보통/어려움)
   - 복잡한 질문을 여러 개의 간단한 하위 질문으로 분해

3. **SQL 생성 (SQL Generation)**
   - 분석된 정보를 바탕으로 실제 SQL 쿼리 작성
   - 복잡도에 따라 적절한 템플릿 사용

4. **자기교정 (Self-Correction)**
   - 생성된 SQL의 문법과 논리 오류 검사 및 수정
   - 데이터베이스 방언(dialect)에 맞게 최적화

### 왜 효과적인가?
- **체계적 접근**: 단계별 처리로 오류 가능성 감소
- **맥락 학습**: 예시를 통한 학습으로 정확도 향상  
- **자동 검증**: 자기교정을 통한 품질 보장

**연구 논문**: [DIN-SQL: Decomposed In-Context Learning of Text-to-SQL with Self-Correction](https://arxiv.org/pdf/2304.11015.pdf)

![Alt text](./images/din_sql_methodology.png)

### 사용할 도구들 소개

이 실습에서는 다음 라이브러리와 도구들을 사용합니다:

- **SQLAlchemy**: 파이썬에서 데이터베이스와 상호작용하기 위한 도구
- **Amazon Bedrock SDK (Boto3)**: AWS의 AI 서비스를 사용하기 위한 도구
- **PyAthena**: Amazon Athena 데이터베이스에 연결하기 위한 도구
- **Jinja2**: 프롬프트 템플릿을 만들기 위한 도구
- **Anthropic Claude**: 자연어 처리를 위한 AI 모델

각 도구의 역할을 이해하면 전체 시스템이 어떻게 작동하는지 파악하기 쉬워집니다.

---

### 1단계: 필수 라이브러리 설치

실습을 시작하기 전에 필요한 모든 라이브러리를 설치합니다. 각 라이브러리의 역할은 다음과 같습니다:

- `sqlalchemy`: 데이터베이스 연결 및 쿼리 실행
- `boto3`: AWS 서비스 (Bedrock, Athena) 연결
- `jinja2`: 프롬프트 템플릿 처리
- `PyAthena`: Amazon Athena 데이터베이스 전용 커넥터
- `pandas`: 데이터 분석 및 표시
- `faiss-cpu`: 벡터 검색 (고급 기능용)

**설치 시 주의사항**: 아래 명령어 실행 중 일부 의존성 충돌 경고가 나타날 수 있지만, 이는 다른 라이브러리와의 버전 충돌로 이번 실습에는 영향을 주지 않으므로 무시하셔도 됩니다.

여기서 이 노트북을 실행하는 데 필요한 모든 의존성을 설치합니다. 이 모듈에서 사용하지 않을 라이브러리들의 의존성 충돌로 인해 발생할 수 있는 **다음 오류들은 무시하셔도 됩니다**:
```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.14.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.5.0 requires faiss-cpu, which is not installed.
amazon-sagemaker-jupyter-scheduler 3.0.4 requires pydantic==1.*, but you have pydantic 2.6.0 which is incompatible.
gluonts 0.13.7 requires pydantic~=1.7, but you have pydantic 2.6.0 which is incompatible.
jupyter-ai 2.5.0 requires pydantic~=1.0, but you have pydantic 2.6.0 which is incompatible.
jupyter-ai-magics 2.5.0 requires pydantic~=1.0, but you have pydantic 2.6.0 which is incompatible.
jupyter-scheduler 2.3.0 requires pydantic~=1.10, but you have pydantic 2.6.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.2 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
```

In [ ]:
!python -m ensurepip --upgrade
%pip install -qU sqlalchemy
%pip install -q "boto3~=1.34"
%pip install -qU jinja2
%pip install -qU botocore
%pip install -qU pandas
%pip install -qU PyAthena
%pip install -qU faiss-cpu

### 필요한 라이브러리 가져오기

이제 DIN-SQL 실습을 위한 라이브러리들을 가져옵니다:

- `boto3`: AWS 서비스 접근
- `pandas`: 데이터 프레임 처리 및 결과 표시
- `din_sql_lib`: DIN-SQL 방법론 구현체 (사전 작성된 라이브러리)
- `utilities`: 워크샵용 유틸리티 함수들

**참고**: `din_sql` 라이브러리는 논문에서 제시된 프롬프트 템플릿을 Jinja2로 구현한 것입니다. 이를 통해 복잡한 프롬프트를 체계적으로 관리할 수 있습니다.

In [ ]:
import sys

import boto3
import pandas as pd

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl
import utilities as u

### 2단계: 데이터베이스 연결 설정

이 단계에서는 Amazon Athena에 저장된 TPC-DS 데이터셋에 연결합니다.

#### TPC-DS 데이터셋이란?
TPC-DS는 데이터 웨어하우스 벤치마크를 위한 표준 데이터셋으로, 소매업체의 판매 데이터를 시뮬레이션합니다. 다음과 같은 테이블들을 포함합니다:
- `customer`: 고객 정보
- `web_sales`: 웹 판매 데이터  
- `catalog_sales`: 카탈로그 판매 데이터
- `item`: 상품 정보
- 기타 관련 테이블들

#### Amazon Athena란?
Athena는 AWS의 서버리스 쿼리 서비스로, S3에 저장된 데이터를 SQL로 직접 분석할 수 있게 해줍니다.

#### 연결 정보 설정

다음 변수들을 CloudFormation 출력값으로 초기화합니다:

- **ATHENA_RESULTS_S3_LOCATION**: 쿼리 결과가 저장될 S3 위치
- **ATHENA_CATALOG_NAME**: Athena 데이터 카탈로그 이름  
- **DB_NAME**: 사용할 데이터베이스 이름 (tpcds1)

이 정보들은 AWS CloudFormation 스택의 출력 탭에서 확인할 수 있습니다.

**CloudFormation이란?** AWS 리소스를 코드로 관리하는 서비스로, 이 워크샵에서는 필요한 모든 AWS 리소스를 자동으로 설정해줍니다.

In [ ]:
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME = \
    u.extract_CF_outputs("AthenaResultsS3Location", "AthenaCatalogName")
# ATHENA_RESULTS_S3_LOCATION = "<workshop bucket name>" # available in cloudformation outputs
# ATHENA_CATALOG_NAME = "<athena catalog name>" # available in cloudformation outputs
# ATHENA_RESULTS_S3_BUCKET = u.extract_s3_bucket(ATHENA_RESULTS_S3_LOCATION)
DB_NAME = "tpcds1"
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME, DB_NAME

#### DIN-SQL 클래스 인스턴스 생성

이제 DIN-SQL 클래스를 생성하고 사용할 AI 모델을 지정합니다.

**Claude V2를 선택하는 이유:**
- DIN-SQL 프롬프트는 Claude V2와 최적화되어 설계되었습니다
- 복잡한 추론과 단계별 사고에 뛰어난 성능을 보입니다
- SQL 생성에 특히 적합한 언어 모델입니다

**Amazon Bedrock이란?** AWS에서 제공하는 완전 관리형 AI 서비스로, 다양한 AI 모델을 API로 간편하게 사용할 수 있게 해줍니다.

In [ ]:
din_sql = dsl.DIN_SQL(bedrock_model_id='anthropic.claude-v2')

#### Athena 데이터베이스 연결

설정한 정보를 사용하여 실제 데이터베이스에 연결합니다.

**이 연결의 용도:**
1. **SQL 테스트**: 생성된 SQL 쿼리가 실제로 실행되는지 확인
2. **스키마 정보 조회**: 테이블과 컬럼 정보를 가져와서 프롬프트에 활용
3. **결과 검증**: 생성된 쿼리의 결과가 올바른지 확인

연결이 성공하면 데이터베이스의 모든 테이블과 관계 정보에 접근할 수 있게 됩니다.

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
                       db_name=DB_NAME, 
                       s3_prefix=ATHENA_RESULTS_S3_LOCATION)

### 3단계: 스키마 연결 (Schema Linking) 

이제 DIN-SQL의 첫 번째 단계인 **스키마 연결**을 시작합니다.

#### 스키마 연결이란?
자연어 질문에 답하기 위해 어떤 데이터베이스 테이블들을 연결해야 하는지 파악하는 과정입니다.

**예시:**  
질문: "어떤 고객이 가장 많은 돈을 썼나요?"
→ 필요한 테이블: `customer` (고객 정보) + `web_sales` (판매 정보)
→ 연결 방법: customer.customer_id = web_sales.customer_id

이 단계가 중요한 이유는 잘못된 테이블을 연결하면 전혀 다른 결과가 나오기 때문입니다.

#### 스키마 연결 프롬프트 살펴보기

먼저 DIN-SQL에서 사용하는 스키마 연결 프롬프트 템플릿을 확인해보겠습니다. 이 프롬프트가 어떻게 설계되었는지 살펴보면 AI가 어떤 방식으로 추론하도록 유도하는지 이해할 수 있습니다.

**프롬프트 설계의 핵심 원칙:**
1. **구체적인 지시사항**: AI에게 정확히 무엇을 해야 하는지 명시
2. **예시 제공**: 다양한 케이스의 예시로 학습 효과 증대  
3. **단계별 사고**: 복잡한 문제를 단계별로 분해하여 해결
4. **구조화된 출력**: XML 태그를 사용해 결과를 명확하게 구분

In [ ]:
!head ../libs/din_sql/prompt_templates/schema_linking_prompt.txt.jinja

In [ ]:
return_sql = din_sql.find_fields(db_name=DB_NAME)
print(return_sql)

#### 프롬프트 설계 분석

위의 프롬프트 템플릿에서 **Anthropic의 프롬프팅 모범 사례**가 어떻게 적용되었는지 분석해보겠습니다:

**1. XML 태그 활용**
```xml
<links>테이블 연결 정보</links>
```
- 프롬프트의 다른 부분들을 명확히 구분
- AI의 출력 형식을 일관되게 유지

**2. Many-shot 학습**
- 많은 예시를 통해 AI가 패턴을 학습하도록 유도
- 다양한 케이스를 다루어 일반화 능력 향상

**3. 단계별 사고 (Chain of Thought)**
- "Let's think step by step" 같은 표현으로 논리적 추론 유도
- 복잡한 문제를 작은 단계로 분해

**4. 역할 부여 (Role-playing)**
- AI를 "관계형 데이터베이스 전문가"로 설정
- 전문적이고 정확한 답변을 유도

이제 실제 질문으로 프롬프트를 만들어보겠습니다.

In [ ]:
question = "Which customer spent the most money in the web store?"

schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)
print(schema_links_prompt)

#### "입에 말 넣어주기" 기법 활용

AI가 답변을 시작할 때 특정 방향으로 유도하는 **"putting words in Claude's mouth"** 기법을 사용합니다.

**이 기법의 장점:**
- AI의 답변 스타일과 구조를 일관되게 유지
- 원하는 형식으로 답변을 유도
- 추론 과정을 더욱 체계적으로 만듦

**예시:**
일반적인 질문: "이 문제를 어떻게 해결할까?"
입에 말 넣어주기: "A. Let's think step by step. In the question..."

이렇게 시작 문장을 제공하면 AI가 더 구조적으로 답변합니다.

In [ ]:
word_in_mouth_schema_link = f'A. Let’s think step by step. In the question "{question}", we are asked:'

#### 스키마 연결 실행

이제 준비된 프롬프트를 Claude에게 전송하여 스키마 연결을 수행해보겠습니다.

**실행 과정:**
1. 질문을 분석하여 필요한 테이블들을 식별
2. 테이블 간의 관계 (Foreign Key) 파악  
3. 적절한 JOIN 조건 결정

**주목할 점:**
- `stop_sequences=['</links>']`: AI가 `</links>` 태그를 만나면 답변을 중단
- 이를 통해 원하는 부분만 정확히 추출 가능

In [ ]:
schema_links = din_sql.llm_generation(
                    schema_links_prompt,
                    stop_sequences=['</links>'],
                    word_in_mouth=word_in_mouth_schema_link
                    )
print(f"{word_in_mouth_schema_link}{schema_links}")

#### 스키마 연결 결과 분석

Claude가 논리적 추론을 통해 테이블 간의 관계를 성공적으로 식별했습니다!

**Claude의 추론 과정 분석:**
1. **질문 분해**: "웹 스토어에서 가장 많은 돈을 쓴 고객" 
2. **필요 정보 식별**: 고객 정보 + 웹 판매 정보
3. **테이블 매핑**: `customer` 테이블 + `web_sales` 테이블
4. **관계 파악**: 외래 키를 통한 테이블 연결 방법 결정

**결과 해석:**
- AI가 데이터베이스의 테이블 구조를 이해하고 있음
- 비즈니스 로직을 SQL 관계로 올바르게 변환
- 구조화된 형태(`<links>` 태그)로 결과 제공

이제 이 연결 정보를 추출하여 다음 단계에서 사용하겠습니다.

In [ ]:
links = u.extract_tag(schema_links+"</links>", "links")[0].strip()
links

### 4단계: SQL 복잡도 분류

이제 DIN-SQL의 두 번째 단계인 **복잡도 분류**를 진행합니다.

#### 복잡도 분류란?
질문에 답하기 위해 필요한 SQL 쿼리의 복잡도를 분석하여 적절한 접근 방법을 선택하는 과정입니다.

**분류 기준:**
- **EASY**: 단일 테이블 쿼리, 간단한 조건
- **MEDIUM (NON-NESTED)**: 여러 테이블 JOIN, 집계 함수 사용
- **HARD (NESTED)**: 서브쿼리, 복잡한 중첩 구조

**왜 분류가 필요한가?**
각 복잡도 수준마다 다른 프롬프트 템플릿과 전략을 사용하여 더 정확한 결과를 얻을 수 있습니다.

프로세스의 다음 단계는 질문에 답하는 데 필요한 SQL의 복잡도를 분류하는 것입니다. 프롬프트를 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/classification_prompt.txt.jinja

#### 분류 프롬프트 분석

분류 프롬프트는 **의사결정 프레임워크**를 제공합니다:

**구조:**
1. **명확한 분류 기준**: 각 복잡도 레벨의 정의
2. **구체적인 예시**: 각 카테고리별 실제 질문과 답변 예시
3. **If-Then 논리**: 명확한 의사결정 규칙

**현재 질문 분석:**
"웹 스토어에서 가장 많은 돈을 쓴 고객은?"
- 여러 테이블 필요 (customer + web_sales)
- 집계 함수 필요 (SUM, MAX)  
- JOIN 연산 필요
→ 예상 분류: **MEDIUM (NON-NESTED)**

이제 Claude가 어떻게 분류하는지 확인해보겠습니다.

In [ ]:
word_in_mouth_classify = "A: Let’s think step by step."
classification = din_sql.llm_generation(
    prompt=din_sql.classification_prompt_maker(question, DB_NAME, links),
    word_in_mouth=word_in_mouth_classify
    )
print(f"{word_in_mouth_classify}{classification}")

#### 분류 결과 분석

Claude가 단계별 사고를 통해 올바른 분류를 수행했습니다!

**Claude의 분류 논리:**
1. **질문 분석**: 고객별 총 구매액 계산 필요
2. **복잡도 평가**: 
   - 여러 테이블 JOIN (customer + web_sales)
   - 집계 함수 사용 (SUM)
   - 정렬 및 최대값 선택
3. **결론**: NON-NESTED (MEDIUM) 복잡도

**분류의 중요성:**
- 이 분류 결과에 따라 다음 단계에서 사용할 프롬프트 템플릿이 결정됩니다
- MEDIUM 복잡도이므로 JOIN과 집계 함수에 특화된 프롬프트를 사용

이제 추출된 분류 정보로 SQL 생성 단계로 넘어가겠습니다.

In [ ]:
predicted_class = u.extract_tag(classification, "label")[0]
predicted_class

### 5단계: SQL 쿼리 생성

이제 DIN-SQL의 핵심인 **SQL 쿼리 생성** 단계입니다!

#### 지금까지의 진행 상황
1. **스키마 연결**: customer ↔ web_sales 테이블 관계 파악
2. **복잡도 분류**: NON-NESTED (MEDIUM) 복잡도 확인
3. **SQL 생성**: 실제 쿼리 작성 (현재 단계)

#### MEDIUM 복잡도 프롬프트의 특징
NON-NESTED 클래스는 `medium_prompt` 템플릿을 사용합니다. 이 템플릿의 특징을 살펴보겠습니다.

질문이 준비되고, 필요한 쿼리의 복잡도가 분류되었으며, 스키마 링크가 식별되었으므로, 이제 SQL 문을 생성할 준비가 되었습니다. 그 전에 프롬프트를 살펴보겠습니다. 'NON-NESTED' 클래스는 'medium_prompt' 템플릿을 사용하므로, 이를 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/medium_prompt.txt.jinja

#### Medium 프롬프트 템플릿 분석

MEDIUM 복잡도 프롬프트의 핵심 특징:

**1. JOIN 중심 예시**
- 여러 테이블을 연결하는 다양한 JOIN 패턴 제공
- INNER JOIN, LEFT JOIN 등 상황별 적절한 JOIN 방법 학습

**2. 집계 함수 활용**
- SUM, COUNT, AVG, MAX, MIN 등 집계 함수 사용법
- GROUP BY와 ORDER BY 절의 올바른 사용

**3. 구조화된 출력**
- ```sql 코드 블록으로 쿼리를 명확히 구분
- 단계별 설명과 함께 최종 쿼리 제공

**4. Stop Sequence 활용**
- `</example>` 태그에서 생성 중단
- 원하는 형식으로만 답변을 제한

이제 실제 SQL을 생성해보겠습니다.

In [ ]:
sql_tag_start = '```sql'
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
sql_qry = din_sql.llm_generation(
                    prompt=din_sql.medium_prompt_maker(
                        test_sample_text=question, 
                        database=DB_NAME, 
                        schema_links=links,
                        sql_tag_start=sql_tag_start,
                        sql_tag_end='```'),
                    stop_sequences=['</example>'],
                    word_in_mouth=word_in_mouth_medium_prompt)
print(f"{word_in_mouth_medium_prompt}{sql_qry}")

#### SQL 생성 결과 분석

훌륭합니다! Claude가 체계적인 사고 과정을 통해 SQL 쿼리를 성공적으로 생성했습니다.

**Claude의 SQL 생성 과정:**
1. **단계별 분석**: 질문을 세부 요구사항으로 분해
2. **테이블 식별**: customer와 web_sales 테이블 필요성 확인
3. **JOIN 설계**: 적절한 외래 키 관계로 테이블 연결
4. **집계 로직**: SUM으로 고객별 총 구매액 계산
5. **정렬 및 제한**: ORDER BY DESC와 LIMIT으로 최고 구매 고객 선택

**생성된 쿼리의 구조:**
- 명확한 테이블 별칭 사용 (c, ws)
- 적절한 JOIN 조건
- 올바른 GROUP BY 절
- 논리적인 ORDER BY와 LIMIT

이제 마지막 SQL 쿼리를 추출하여 테스트해보겠습니다.

In [ ]:
SQL = sql_qry.split('```')[0]
print(f"{SQL}")

### 6단계: 쿼리 실행 및 결과 확인

이제 생성된 SQL 쿼리를 실제 데이터베이스에서 실행해보겠습니다!

#### 실행 과정
1. **쿼리 실행**: SQLAlchemy를 통해 Athena에서 쿼리 실행
2. **결과 변환**: 결과를 Pandas DataFrame으로 변환하여 읽기 쉽게 표시
3. **결과 해석**: 데이터가 우리 질문에 올바르게 답하는지 확인

#### 예상 결과
- 고객 ID (c_customer_sk)
- 고객 이름 (c_first_name, c_last_name)  
- 총 구매액 (계산된 값)

실제 결과를 확인해보겠습니다.

쿼리를 테스트하여 결과가 우리가 예상한 것과 일치하는지, 그리고 실제로 우리 질문에 답하는지 확인해보겠습니다. SQL Alchemy 결과 집합을 반환하고 Pandas 데이터프레임을 사용하여 상호작용하는 방식으로 진행하겠습니다.

In [ ]:
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

### 7단계: 결과 검증

생성된 쿼리가 올바른 답을 제공하는지 검증해보겠습니다.

#### 검증 방법
별도의 검증 쿼리를 작성하여 상위 10명의 고객을 조회하고, 우리가 생성한 쿼리의 결과와 비교해보겠습니다.

**검증 쿼리의 특징:**
- 동일한 비즈니스 로직 (고객별 웹 판매 총액)
- 더 많은 결과 (TOP 10) 조회하여 순위 확인
- 명시적인 필드명 사용으로 명확성 확보

이를 통해 AI가 생성한 쿼리의 정확성을 객관적으로 평가할 수 있습니다.

In [ ]:
validation_query = """
    SELECT "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    , SUM("ws"."ws_ext_list_price") as total_sales
    FROM "customer" "c" 
    JOIN "web_sales" "ws" 
        ON "ws"."ws_bill_customer_sk" = "c"."c_customer_sk"   
    GROUP BY "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    ORDER BY total_sales desc
    limit 10
"""
validation_set = din_sql.query(validation_query)
pd.DataFrame(validation_set)

#### 검증 결과 분석

두 쿼리의 결과를 비교해보세요:

**비교 포인트:**
1. **고객 ID 일치**: 두 결과 모두 동일한 고객 SK가 1위에 나타나는가?
2. **사용 필드 차이**: 
   - 생성된 쿼리: 어떤 필드를 사용했나요?
   - 검증 쿼리: `ws_ext_list_price` (상품 정가) 사용
3. **비즈니스 로직**: 실제 "구매액"은 정가인가, 실제 지불 금액인가?

**만약 결과가 다르다면:**
- 필드 선택의 차이 (list_price vs sales_price vs net_paid 등)
- 비즈니스 정의의 모호함 ("구매액"의 정확한 의미)
- 프롬프트 개선 필요성

만약 쿼리에서 오류가 발생했다면, 다음 단계인 **자기교정**으로 넘어가겠습니다.

### 8단계: 자기교정 (Self-Correction)

DIN-SQL의 마지막 단계인 **자기교정**을 진행합니다.

#### 자기교정이란?
생성된 SQL 쿼리에서 발생할 수 있는 다음과 같은 문제들을 AI가 스스로 검토하고 수정하는 과정입니다:

**검토 항목:**
1. **문법 오류**: SQL 구문 오류 확인
2. **논리 오류**: 비즈니스 로직의 정확성 확인  
3. **데이터베이스 방언**: 특정 DB 엔진(Presto, MySQL 등)에 맞는 구문 사용
4. **성능 최적화**: 더 효율적인 쿼리로 개선

#### 왜 자기교정이 중요한가?
- AI가 생성한 첫 번째 답안이 항상 완벽하지는 않음
- 데이터베이스별 문법 차이 해결
- 미묘한 논리 오류 수정

자기교정 프롬프트를 살펴보겠습니다.

이는 프로세스의 마지막 단계입니다. 주어진 SQL 문에서 코드에 잘못된 부분이 있다면 수정하도록 Claude에게 요청하여 SQL 코드를 마지막으로 한 번 더 확인합니다. 이것이 어떻게 수행되는지 지시사항을 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/clean_query_prompt.txt.jinja

#### 자기교정 실행

이제 Athena가 사용하는 **Presto SQL 방언**에 맞게 쿼리를 검토하고 수정하겠습니다.

**Presto SQL의 특징:**
- 큰따옴표로 테이블/컬럼명 감싸기
- 특정 함수나 구문의 차이
- 데이터 타입 처리 방식

**자기교정 프로세스:**
1. 원본 쿼리를 Presto 문법으로 검토
2. 문법 오류나 개선점 식별
3. 수정된 쿼리 제공
4. 변경사항에 대한 설명

Claude가 어떤 수정사항을 제안하는지 확인해보겠습니다.

In [ ]:
revised_sql = din_sql.debugger_generation(
            prompt=din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
            ).replace("\n", " ")
print(f"{revised_sql}")

#### 자기교정 결과 처리

Claude가 자기교정을 완료했습니다. 이제 수정된 SQL 쿼리를 추출해보겠습니다.

**자기교정에서 확인할 수 있는 개선사항:**
- **문법 정확성**: Presto SQL 문법에 맞는 수정
- **인용부호 사용**: 테이블/컬럼명의 올바른 인용
- **함수 사용법**: 데이터베이스별 함수 문법 준수
- **성능 개선**: 더 효율적인 쿼리 구조

수정된 최종 쿼리를 확인해보겠습니다.

In [ ]:
SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

### 9단계: 추가 실험

#### 첫 번째 질문의 완료
축하합니다! DIN-SQL의 전체 4단계 프로세스를 성공적으로 완료했습니다:

1. **스키마 연결**: 필요한 테이블 관계 파악
2. **복잡도 분류**: 적절한 접근 방법 선택  
3. **SQL 생성**: 실제 쿼리 작성
4. **자기교정**: 오류 수정 및 최적화

#### 학습 성과 평가
- **정확성**: 생성된 쿼리가 예상한 결과를 제공했나요?
- **효율성**: 쿼리가 논리적이고 최적화되어 있나요?
- **일관성**: 다른 질문에도 동일한 방법이 적용될 수 있을까요?

#### 개선 방향 고민
결과가 예상과 다르다면:
- 프롬프트를 어떻게 개선할 수 있을까요?
- 더 명확한 비즈니스 정의가 필요한가요?
- 추가적인 검증 단계가 필요한가요?

이제 두 번째 질문으로 전체 프로세스를 다시 실행해보겠습니다.

#### 두 번째 질문으로 전체 프로세스 재실행

이제 새로운 질문 **"어떤 연도에 카탈로그 판매량이 가장 높았나요?"**로 DIN-SQL 프로세스를 처음부터 끝까지 실행해보겠습니다.

**이 실험의 목적:**
1. **일관성 검증**: DIN-SQL 방법론이 다른 유형의 질문에도 적용되는지 확인
2. **학습 강화**: 동일한 프로세스를 반복하여 이해도 향상
3. **패턴 인식**: 다양한 질문 유형별 접근 방법 학습

**새로운 질문의 특징:**
- 시간 축 분석 (연도별)
- 다른 테이블 사용 (catalog_sales)
- 집계 함수 사용 (판매량 합계)

아래 코드가 4단계를 모두 자동으로 실행합니다.

In [ ]:
question = 'What year had the highest catalog sales?'

#get schema links
schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)
schema_links = din_sql.llm_generation(
                    schema_links_prompt,
                    stop_sequences=['</links>'],
                    word_in_mouth=word_in_mouth_schema_link)
print(f"{word_in_mouth_schema_link}{schema_links}")
print(schema_links)
links = schema_links.split('<links>')[1].replace('\n','')

# classify and decompose
word_in_mouth_classify = "A: Let’s think step by step."
classification = din_sql.llm_generation(
    prompt=din_sql.classification_prompt_maker(question, DB_NAME, links),
    word_in_mouth=word_in_mouth_classify
    )
print(f"{word_in_mouth_classify}{classification}")
predicted_class = classification.split("<label>")[1].split("</label>")[0].strip()

# generate SQL
sql_tag_start = '```sql'
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
sql_qry = din_sql.llm_generation(
                    prompt=din_sql.medium_prompt_maker(
                        test_sample_text=question, 
                        database=DB_NAME, 
                        schema_links=links,
                        sql_tag_start=sql_tag_start,
                        sql_tag_end='```'),
                    stop_sequences=['</example>'],
                    word_in_mouth=word_in_mouth_medium_prompt
                    )
print(f"{word_in_mouth_medium_prompt}{sql_qry}")
SQL = sql_qry.split('```')[0].strip()

In [ ]:
pd.DataFrame(din_sql.query(SQL))

#### 두 번째 질문 실행 중 오류 발생 시 대응

만약 생성된 쿼리에서 오류가 발생한다면, 이는 DIN-SQL에서 자기교정 단계의 중요성을 보여주는 좋은 예시입니다.

**일반적인 오류 원인:**
1. **날짜 필드 처리**: 연도 추출 함수의 차이 (YEAR() vs EXTRACT())
2. **데이터 타입 변환**: 문자열을 날짜로 변환하는 방법
3. **테이블 구조**: 예상과 다른 테이블 스키마
4. **NULL 값 처리**: 빈 데이터에 대한 처리 방식

**해결 방법:**
자기교정 단계를 통해 이러한 오류들을 자동으로 수정하고 최적화할 수 있습니다.

오류가 발생했다면 다음 셀에서 자기교정을 실행해보겠습니다.

In [ ]:
# self correction
revised_sql = din_sql.debugger_generation(
            prompt=din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
            ).replace("\n", " ")
print(f"{revised_sql}")
SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

# see results
result_set = pd.DataFrame(din_sql.query(SQL))
result_set

### 참고문헌 및 추가 학습 자료

#### 원논문 및 코드
```
@article{pourreza2023din,
  title={DIN-SQL: Decomposed In-Context Learning of Text-to-SQL with Self-Correction},
  author={Pourreza, Mohammadreza and Rafiei, Davood},
  journal={arXiv preprint arXiv:2304.11015},
  year={2023}
}
```

**논문**: [https://arxiv.org/abs/2304.11015](https://arxiv.org/abs/2304.11015)  
**원본 코드**: [https://github.com/MohammadrezaPourreza/Few-shot-NL2SQL-with-prompting](https://github.com/MohammadrezaPourreza/Few-shot-NL2SQL-with-prompting)

#### 추가 학습 자료
- **Anthropic 프롬프팅 가이드**: Claude와 효과적으로 상호작용하는 방법
- **SQL 최적화**: 다양한 데이터베이스 방언에 대한 이해
- **Text-to-SQL 벤치마크**: Spider, WikiSQL 등 표준 데이터셋

#### 실무 적용 방안
- **데이터 분석 자동화**: 비개발자도 쉽게 데이터 조회
- **BI 도구 개선**: 자연어 인터페이스 구축
- **고객 지원**: 자동화된 데이터 조회 서비스

### 추가 정보: 다른 복잡도 레벨 사용법

다른 복잡도 레벨의 프롬프트를 사용해야 하는 경우를 위한 참고 정보입니다.

#### HARD (복잡한 중첩 쿼리) 프롬프트
복잡한 서브쿼리가 필요한 질문의 경우 `hard_prompt_maker`와 함께 사용:

```python
word_in_mouth_hard_prompt = f'''A: Let's think step by step. "{question}" can be solved by knowing the answer to the following sub-question "{{sub_questions}}".
The SQL query for the sub-question "'''
```

#### EASY (단순 쿼리) 프롬프트  
단일 테이블의 간단한 조회인 경우 `easy_prompt_maker`와 함께 사용:

```python
word_in_mouth_easy_prompt = f"SQL: ```sql"
```

#### 사용 시기
- **EASY**: 단일 테이블, 간단한 필터링, 기본 정렬
- **MEDIUM**: 여러 테이블 JOIN, 집계 함수, GROUP BY
- **HARD**: 서브쿼리, 윈도우 함수, 복잡한 중첩 구조

각 복잡도에 맞는 적절한 프롬프트를 사용하면 더 정확한 결과를 얻을 수 있습니다.

In [ ]:
word_in_mouth_hard_prompt = f'''A: Let's think step by step. "{question}" can be solved by knowing the answer to the following sub-question "{{sub_questions}}".
The SQL query for the sub-question "'''
word_in_mouth_easy_prompt = f"SQL: {sql_tag_start}"